<h1> A custome notebook for every single keras built in method </h1>
<h2> This notebook is a collection of all the keras built in methods and their usage </h2>
<ol> 
<h2>List of Custom models</h2>
<li> custome loss function </li>
<li> custome activation function </li>
<li> custome regularizer </li>
<li> custome initializer </li>
<li> custome constraint </li>
<li> custome metric </li>
<li> custome layer </li>
<li> custome callback </li>
<li> custome optimizer </li>
<li> custome preprocessing layer </li>
<li> custome training loop </li>
<li> custome training step </li>
<li> custome training </li>
</ol>

In [4]:
# custome loss function using keras backend
import tensorflow as tf
class CustomizedLoss(tf.keras.losses.Loss):
    def __init_self(self,threshold=.1,**kwargs):
        self.threshold=threshold
        super().__init__(**kwargs)

    def call(self,y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<self.threshold
        squared_error = tf.square(error)/2
        linear_error = self.threshold*tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,squared_error,linear_error)
    def get_config(self):
        base = super().get_config()
        return {**base,"threshold":self.threshold}  # to add threshold to the config
    

In [7]:
# custom activation function
class CustomizedActivation(tf.keras.layers.Layer):
    def __init__(self,activation,**kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
    def call(self,inputs):
        return self.activation(inputs)
    def get_config(self):
        base = super().get_config()
        return {**base,"activation":tf.keras.activations.serialize(self.activation)}

In [8]:
# custom initializer
class CustomizedInitializer(tf.keras.initializers.Initializer):
    def __init__(self,factor,**kwargs):
        self.factor = factor
        super().__init__(**kwargs)
    def __call__(self,shape,dtype=None):
        return tf.fill(shape,self.factor)
    def get_config(self):
        base = super().get_config()
        return {**base,"factor":self.factor}


In [9]:
# custom regularizer
class customized_l1_regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self) -> None:
        super().__init__()

In [10]:
# custom constraint
class CustomizedConstraint(tf.keras.constraints.Constraint):
    def __init__(self,factor,**kwargs):
        self.factor = factor
        super().__init__(**kwargs)
    def __call__(self,weight):
        return weight + self.factor
    def get_config(self):
        base = super().get_config()
        return {**base,"factor":self.factor}
    

In [11]:
# custom metric
class CustomizedMetric(tf.keras.metrics.Metric):
    def __init__(self,threshold = 0.1, **kwargs):
        self.threshold = threshold
        self.huber_loss = CustomizedLoss(threshold)
        self.total = self.add_weight("total",initializer="zeros")
        self.count = self.add_weight("count",initializer="zeros")
        super().__init__(**kwargs)
    def update_state(self,y_true,y_pred,sample_weight=None):
        loss = self.huber_loss(y_true,y_pred)
        self.total.assign_add(tf.reduce_sum(loss))
        self.count.assign_add(tf.cast(tf.size(y_true),tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base = super().get_config()
        return {**base,"threshold":self.threshold}
    

In [ ]:
# custom layer
class CustomizedLayer(tf.keras.layers.Layer):
    def __init__(self,units,activation=None,**kwargs):
        self.activation = tf.keras.activations.get(activation)
        self.units = units
        super().__init__(**kwargs)
    def build(self,batch_input_shape):
        self.kernal = self.add_weight(
            name="kernal",shape=[batch_input_shape[-1],self.units],
            initializer="glorot_normal"
        )
        self.bias = self.add_weight(
            name="bias",shape=[self.units],initializer="zeros"
        )
        super().build(batch_input_shape)
    def call(self,X)
        return self.activation(
            X @ self.kernal + self.bias
        )
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1]+[self.units])
    def get_config(self):
        base = super().get_config()
        return {**base,"units":self.units,"activation":tf.keras.activations.serialize(self.activation)}
    
